In [1]:
%matplotlib inline
from sklearn import datasets, model_selection, tree, metrics, ensemble, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import seaborn
import numpy as np

# Задание 1. (4 балла)


Вам нужно реализовать градиентный бустинг для задачи регрессии на датасете boston.

Заполните кодом строчки в TODO. Запустите, сделайте предсказание, проверьте что все ок, и выведите rmse ошибку.

In [2]:
data = datasets.load_boston()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [3]:
X = data['data']
y = data['target']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=41)

In [5]:
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeRegressor
import numpy as np


# Параметрами с которыми вы хотите обучать деревья
TREE_PARAMS_DICT = {'max_depth': 1}
# Параметр tau (learning_rate) для вашего GB
TAU = 0.05
ITERS = 100


class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters, tau):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.base_algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, y_data)
        self.estimators = []
        curr_pred = self.base_algo.predict(X_data)
        for iter_num in range(self.iters):
            # Нужно посчитать градиент функции потерь
            # For regression L(y, a) = 0.5 * (a-y)**2
            #.               L'(y, a) = a - y ==> antigradient = y - a, where a = curr_pred
            grad = curr_pred - y_data# TODO
            # Нужно обучить DecisionTreeRegressor предсказывать антиградиент
            # Не забудьте про self.tree_params_dict
            algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, -grad) # TODO

            self.estimators.append(algo)
            # Обновите предсказания в каждой точке
            # Just add contribution of the current tree to the predictions:
            curr_pred += self.tau * algo.predict(X_data) # TODO
        return self
    
    def predict(self, X_data):
        # Предсказание на данных
        res = self.base_algo.predict(X_data)
        for estimator in self.estimators:
            res += self.tau * estimator.predict(X_data)
        
        return res

Let's run the model:

In [6]:
simple_gb = SimpleGB(TREE_PARAMS_DICT, ITERS, TAU)

In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

In [8]:
- np.mean(cross_val_score(simple_gb, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error'))

5.135524138817064

Let's check the results on the test set:

In [9]:
simple_gb.fit(X_train, y_train)
y_pred = simple_gb.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred))

5.671875533154961

Let's compare with Linear Regression (just to see if the `rmse` score above is within the norm):

In [10]:
from sklearn.linear_model import LinearRegression

In [11]:
lr = LinearRegression()

In [12]:
- np.mean(cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_root_mean_squared_error'))

5.1027151819431396

In [13]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred))

4.597800668458814

It works (:

# Задание 2. (1 балл)

Датасет маленький, давайте возьмем что-нибудь поинтереснее. Например, HR.csv (лежит в папке с занятием 30_11/trees).

Подгрузите датасет, сделайте split на train/test, random_state = 42. Выберите любую модель - xgboost, lightgbm, catboost. И проведите несколько экспериментов.

Подберите оптимальные с точки зрения метрики качества параметры алгоритмов, например:
* глубину
* количество деревьев

Посмотрите в документации, с какими еще параметрами можно поиграться. Но не делайте слишком большой поиск - будет долго считаться. 

Обучите несколько различных моделей, сделайте блендинг и сравните качество. 


In [14]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [15]:
df = pd.read_csv('HR.csv')

In [16]:
df.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [17]:
df.shape

(14999, 7)

In [18]:
df['promotion_last_5years'].unique()

array([0, 1])

In [19]:
X = df.iloc[:, :-1].copy()
y = df.iloc[:, -1].copy()

#### Observation:
- We see that the dataset is highly unbalanced:

In [20]:
y.mean()

0.021268084538969265

So we make train-test split and cross-validation accordingly:

In [21]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, stratify=y, random_state=41)

In [22]:
xgb_clf = XGBClassifier()

In [23]:
params_grid = {
    'n_estimators': range(50, 200, 50),
    'max_depth': [4,6,8, 10],
    'colsample_bylevel': [0.4,0.6,0.8, 1.0]
}

In [24]:
# Take data imbalanced into account for grid search
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=41)
grid_search = GridSearchCV(xgb_clf, params_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)

In [25]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=41, shuffle=True),
             estimator=XGBClassifier(), n_jobs=-1,
             param_grid={'colsample_bylevel': [0.4, 0.6, 0.8, 1.0],
                         'max_depth': [4, 6, 8, 10],
                         'n_estimators': range(50, 200, 50)},
             scoring='neg_log_loss')

In [26]:
cv_results = pd.DataFrame(grid_search.cv_results_)[['param_colsample_bylevel', 'param_max_depth', 'param_n_estimators', 'mean_test_score', 'rank_test_score']].sort_values(by='rank_test_score')
cv_results.rename(columns={'mean_test_score': 'mean_test_neg_log_loss'}, inplace=True)
cv_results.columns = cv_results.columns.str.replace('param_', '')

cv_results.head()

,colsample_bylevel,max_depth,n_estimators,mean_test_neg_log_loss,rank_test_score
45,1,10,50,-0.086522,1
34,0.8,10,100,-0.086918,2
43,1,8,100,-0.087068,3
33,0.8,10,50,-0.087218,4
46,1,10,100,-0.087237,5


#### Observations:

- We see that the best set of parameters are those that:
  - have the deepest trees (`max_depth` - $8$ and $10$).
  - use most of the features when doing split at each level (`colsample_bylevel` - $80\%$ and $100\%$ of the features).

Try blending (model comparison at the end):

In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [28]:
class Blender(BaseEstimator):
  def __init__(self, estimators, meta_estimator):
    self.estimators = estimators
    self.meta_estimator = meta_estimator
    self.predictions_by_model = dict()

  
  def fit(self, X, y):

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=41)

    X_val_meta = X_val

    for model_name, estimator in self.estimators.items():

      estimator.fit(X_train, y_train)
      pred = estimator.predict(X_val)

      X_val_meta = np.c_[X_val_meta, pred]
    

    self.meta_estimator.fit(X_val_meta, y_val)


    
  def predict(self, X):

    X_test_meta = X

    for model_name, estimator in self.estimators.items():
      pred = estimator.predict(X)
      X_test_meta = np.c_[X_test_meta, pred]

      self.predictions_by_model[model_name] = pred
    
    y_pred = self.meta_estimator.predict(X_test_meta)

    return y_pred

In [29]:
estimators = {
    'dtree': DecisionTreeClassifier(),
    'knn' : KNeighborsClassifier(),
    'svc' : SVC()
}

meta_estimator = LogisticRegression(max_iter=500)

In [30]:
blender = Blender(estimators=estimators, meta_estimator=meta_estimator)

In [31]:
blender.fit(X_train, y_train)

Let's compare the models. (For curiosity, for this highly imbalanced dataset we'll use weighted precision, recall and f_beta scores):

In [32]:
from sklearn.metrics import precision_recall_fscore_support

In [33]:
def score_model(y_true, y_pred):
  precision, recall, f_beta, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted', labels=np.unique(y_pred))

  scores = {
      'precision': precision,
      'recall': recall,
      'f_beta': f_beta
  }
  
  return scores

In [34]:
scores_by_models = {}

# XGBClassifier
best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)

scores_by_models['xbg_clf'] = score_model(y_test, y_pred)

# Custum Blender
y_pred = blender.predict(X_test)
scores_by_models['blender'] = score_model(y_test, y_pred)


# Base estimators in custom blender
for model_name, y_pred in blender.predictions_by_model.items():
  scores_by_models[model_name] = score_model(y_test, y_pred)

In [35]:
pd.DataFrame.from_dict(scores_by_models).transpose().sort_values(by=['precision', 'recall', 'f_beta'], ascending=False)

,precision,recall,f_beta
xbg_clf,0.981169,0.980800,0.973039
svc,0.978667,1.000000,0.989218
dtree,0.973437,0.969600,0.971392
blender,0.972516,0.978933,0.969271
knn,0.968955,0.978667,0.969125


In [36]:
y_test.mean()

0.021333333333333333

#### Conclusions:
- These results are obtained, just from one run of the algorithms. A more meaningful results requires averaging over repeated experiments results.

- We see that (as expected, I guess) XGBClassifier performs better (with respect to `precision`) that our custom blending ensemble and any other of the blender's base models alone.
- Our custom blender performs worse than some its the base estimators. This may be related to the fact that:
  - The `knn` base estimator performs comparatively worse than `svc`, and `dtree`, which may lead the blender to find some compromise between the performance of the base models, and choose something _between them_.

- **Important:** The results above may be misleading due to the highly imbalance of the dataset labels ($\approx 2\%$ of the instances are from the positive class). It may be the case that, even though we split train and test set according to this distribution, the models may have predicted only values from the negative class, for which case the label in `y_test` that are not in `y_pred` (labels $1$) are ignored when computing `precision`, `recall` and `f_beta`.